In [ ]:
# Question: Do detached() tensors track their own gradients seperately?
# Ans: Yes!
# https://discuss.pytorch.org/t/why-is-the-clone-operation-part-of-the-computation-graph-is-it-even-differentiable/67054/11

import torch

a = torch.tensor([2.0], requires_grad=True)
b = a.detach()
b.requires_grad = True

la = (5.0 - a)**2
la.backward()
print(f'a.grad = {a.grad}')

lb = (6.0 - b)**2
lb.backward()
print(f'b.grad = {b.grad}')

In [ ]:
import torch
import torch.nn as nn

from collections import OrderedDict

params = OrderedDict([
    ('fc0', nn.Linear(in_features=4,out_features=4)),
    ('ReLU0', nn.ReLU()),
    ('fc1', nn.Linear(in_features=4,out_features=1))
])
mdl = nn.Sequential(params)

print(params)
print(mdl._parameters)
print(params == params)
print(mdl._parameters == params)
print(mdl._modules)

print()
for name, w in mdl.named_parameters():
    print(name, w.norm(2))

print()    
#mdl._modules['fc0'] = nn.Linear(10,11)
mdl._modules[0]

for name, w in mdl.named_parameters():
    print(name, w.norm(2))

In [ ]:
## Q: are parameters are in computation graph?
import torch
import torch.nn as nn
from torchviz import make_dot

from collections import OrderedDict

fc0 = nn.Linear(in_features=3,out_features=1)
params = [('fc0', fc0)]
mdl = nn.Sequential(OrderedDict(params))

x = torch.randn(1,3)
y = torch.randn(1)

l = ( mdl(x) - y )**2

# make_dot(l,{x:'x',y:'y','fc0':fc0})
print(fc0.weight)
print(fc0.bias)
print(fc0.weight.to_tens)
print()
# make_dot(l,{x:'x',y:'y','fc0':fc0})
make_dot(l,{'x':x,'y':y})
make_dot(l)

In [ ]:
'''
expand
'''

import torch

x = torch.randn([2,3,4,5])

# h_0 of shape (num_layers * num_directions, batch, hidden_size)
h = torch.randn([1,4,8])

x_mean = x.mean()
print(x_mean.size())
print(x_mean)
x = x_mean.expand_as(h)
print(x.size())
print(x)

In [ ]:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
type(device)
print(device == 'cpu')
device.type

In [ ]:
# THIS WORKS

from torch.utils.tensorboard import SummaryWriter

from pathlib import Path


#log_dir (string) – Save directory location. 
#Default is runs/CURRENT_DATETIME_HOSTNAME, which changes after each run.

tb = SummaryWriter()
tb.add_scalar('loss', 111)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

from pathlib import Path


def CURRENT_DATETIME_HOSTNAME(comment=''):
    #if not log_dir:
    import socket
    import os
    from datetime import datetime
    current_time = datetime.now().strftime('%b%d_%H-%M-%S')
    log_dir = os.path.join('runs', current_time + '_' + socket.gethostname() + comment)
    return Path(log_dir)

#log_dir (string) – Save directory location. 
#Default is runs/CURRENT_DATETIME_HOSTNAME, which changes after each run.
# tensorboard --logdir=runs
log_dir = (Path('~/automl-meta-learning/') / CURRENT_DATETIME_HOSTNAME()).expanduser()
print(log_dir)
tb = SummaryWriter(log_dir=log_dir)
tb.add_scalar('loss', 15)

In [ ]:
# download mini-imagenet automatically

#from torchvision.utils import download_and_extract_archive

import torchvision.utils as utils

print(utils)
#print(download_and_extract_archive)

In [ ]:
#torch concat, https://pytorch.org/docs/stable/torch.html#torch.cat
# Concatenates the given sequence of seq tensors in the given dimension. 
# All tensors must either have the same shape (except in the concatenating dimension) or be empty.
import torch

g1 = torch.randn(3,2)
g2 = torch.randn(4,2)

g3 = torch.randn(4,2,3)

grads = [g1, g2]
print(g1.view(-1).size())
print(g2.view(-1).size())
print(g3.view(-1).size())
#print(g3.view(-1))

grads = torch.cat(grads, dim=0)
print(grads)
print(grads.size())
print(grads.mean())
print(grads.std())

# torch stack, https://pytorch.org/docs/stable/torch.html#torch.stack
# Concatenates sequence of tensors along a new dimension. 
# All tensors need to be of the same size.
# torch.stack([g1,g2], dim=0)

In [ ]:
import torch

a = torch.tensor([1,2,3.], requires_grad=True)
a_detached = a.detach()
print(a_detached.is_leaf)
a_detached_sum = a.sum()
print(c.is_leaf)
d = c.detach()
print(d.is_leaf)

In [ ]:
import torch

from types import SimpleNamespace
from pathlib import Path
from pprint import pprint

x = torch.empty([1,2,3])
print(x.size())

args = SimpleNamespace()
args.data_root = "~/automl-meta-learning/data/miniImagenet"

#n1313361300001299.jpg
args.data_root = Path(args.data_root).expanduser()

In [ ]:
import torch

CHW = 3,12,12
x = torch.randn(CHW)
y = torch.randn(CHW)

new = [x,y]
new = torch.stack(new)
print(x.size())
print(new.size())

In [ ]:
print('a');print('b')

In [ ]:
# conver list to tensor

import torch

x = torch.tensor([1,2,3.])
print(x)

In [ ]:
    from torchvision.transforms import Compose, Resize, ToTensor

    import torchmeta
    from torchmeta.datasets.helpers import miniimagenet

    from pathlib import Path
    from types import SimpleNamespace

    from tqdm import tqdm

    ## get args
    args = SimpleNamespace(episodes=5,n_classes=5,k_shot=5,k_eval=15,meta_batch_size=1,n_workers=4)
    args.data_root = Path("~/automl-meta-learning/data/miniImagenet").expanduser()

    ## get meta-batch loader
    train_transform = Compose([Resize(84), ToTensor()])
    dataset = miniimagenet( 
        args.data_root, 
        ways=args.n_classes, 
        shots=args.k_shot, 
        test_shots=args.k_eval,
        meta_split='train',
        download=False)
    dataloader = torchmeta.utils.data.BatchMetaDataLoader(
        dataset, 
        batch_size=args.meta_batch_size,
        num_workers=args.n_workers)

    with tqdm(dataset):
        print(f'len(dataloader)= {len(dataloader)}')
        for episode, batch in enumerate(dataloader):
            print(f'episode = {episode}') 
            train_inputs, train_labels = batch["train"]
            print(f'train_labels[0] = {train_labels[0]}')
            print(f'train_inputs.size() = {train_inputs.size()}')
            pass
            if episode >= args.episodes:
                break

In [9]:
# zip tensors

import torch

x = torch.tensor([1.,2.,3.])
y = torch.tensor([1,2,3])

print(list(zip(x,y)))

xx = torch.randn(2,3,84,84)
yy = torch.randn(2,3,32,32)

print(len(list(zip(xx,yy))))

[(tensor(1.), tensor(1)), (tensor(2.), tensor(2)), (tensor(3.), tensor(3))]
2
